In [15]:
GYRO = True
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 15
MODEL_NAME = f"epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 8
SAVE_MODEL = True
TEST_SIZE = 8
NORMALISE = False
LEAVE_ONE_OUT = False
OVERLAP_ON_TEST_SET = False


### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [16]:
import helpers.file_tagger as file_tagger
import helpers.sequence_generator as sequence_generator
import helpers.split_by_student as split_by_student

import tensorflow as tf
import numpy as np
from keras import layers, Sequential
import tensorboard
import time

ALL_ACTIVITIES = [
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
]

POSSIBLE_OUTCOMES = [
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
]


DATA_DIRECTORY = "./all_respeck"
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}

if OVERLAP_ON_TEST_SET:
    TEST_SEQUENCE_OVERLAP = SEQUENCE_OVERLAP
else:
    TEST_SEQUENCE_OVERLAP = 0

In [17]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        activity = key.split("&")[0]
        if activity in ALL_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if csv_file in file_names:
                    if gyro:
                        sequences = sequence_generator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)
                    else:
                        sequences = sequence_generator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)

                    # iterate through each generated sequence
                    for sequence in sequences:
                        if activity == "standing" or activity == "sitting":
                            tagged_data.append(("sitting_or_standing", sequence))
                        else:
                            tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [18]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [19]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [20]:
def leave_one_out():
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([], [])) #batch_size, epochs
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))
        print("number of students tested so far: " + str(len(test_accuracies)))
        time.sleep(3)

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [21]:
if LEAVE_ONE_OUT:
    leave_one_out()
    exit()

train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs



# Evaluate the model on the test set

test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# print("labels by index:")
# for label, index in LABEL_TO_INDEX.items():
#     print(f"{index}: {label}")

if SAVE_MODEL:
# Save the trained model
    model.save(f"models/Task1/task1_GYRO:{GYRO}_ACCURACY:{test_accuracy}.keras")

Train Set: s56, s67, s74, s79, s32, s7, s40, s98, s92, s9, s22, s65, s59, s82, s42, s77, s27, s71, s15, s17, s87, s100, s16, s13, s60, s91, s61, s86, s55, s30, s48, s45, s93, s64, s88, s102, s63, s12, s66, s21, s80, s72, s84, s43, s83, s29, s39, s38, s11, s96, s54, s57, s34, s36, s50
Dev Set: s5, s44, s33, s95, s23, s51, s52, s1
Test Set: s18, s46, s97, s35, s75, s8, s3, s70
there are 31155 tagged sequences in the dataset
there are 2075 tagged sequences in the dataset
there are 2098 tagged sequences in the dataset
Epoch 1/15
3116/3116 [==============================] - 19s 6ms/step - loss: 0.4654 - accuracy: 0.8712 - val_loss: 0.4207 - val_accuracy: 0.8795
Epoch 2/15
3116/3116 [==============================] - 18s 6ms/step - loss: 0.2581 - accuracy: 0.9251 - val_loss: 0.4093 - val_accuracy: 0.9031
Epoch 3/15
3116/3116 [==============================] - 20s 6ms/step - loss: 0.2036 - accuracy: 0.9453 - val_loss: 0.4105 - val_accuracy: 0.9080
Epoch 4/15
3116/3116 [=======================